<p>This notebook presents a <code>regime_switching</code> module usage scenario, based on chapter 5.4.2 from Kim and Nelson and Kim's code sample http://econ.korea.ac.kr/~cjkim/MARKOV/programs/kim_je.opt.</p>
<p>Lam's Generalized Hamilton Model has the following notation:</p>
<p>$$ y_t = n_t + x_t $$</p>
<p>$$ n_t = n_{t-1} + \delta_{S_t} $$</p>
<p>$$ \phi(L) x_t = x_t - \phi_1 x_{t-1} - \phi_2 x_{t-2} = u_t $$</p>
<p>$$ u_t \sim i.i.d. N(0, \sigma^2) $$</p>
<p>$$ \delta_{S_t} = \delta_0 + \delta_1 S_t $$</p>
<p>$$ Pr[S_t = 1 \mid S_{t-1} = 1] = p, Pr[S_t = 0 \mid S_{t-1} = 0] = q $$</p>
We assume, that roots of equation $ \phi(x) = 0 $ lie inside the unit circle.<br>
A state-space representation for this model:
<p>$$ \Delta y_t = [1 -1] \begin{bmatrix} x_t \\ x_{t-1} \end{bmatrix} + \delta_{S_t} $$</p>
<p>$$ \begin{bmatrix} x_t \\ x_{t-1} \end{bmatrix} = \begin{bmatrix} \phi_1 & \phi_2 \\ 1 & 0 \end{bmatrix} \begin{bmatrix} x_{t-1} \\ x_{t-2} \end{bmatrix} + \begin{bmatrix} u_t \\ 0 \end{bmatrix} $$</p>
<p>The code presented below shows maximum likelihood estimation for this model in two ways, depending on starting parameters for optimization.</p>
<ul>
<li>The first way is to start with some apriori known parameters, which are close to the global maximum of likelihood. This approach and parameter values are taken from the code, referenced above.</li>
<li>The second is to estimate parameters for natural non-switching analog (i.e. when St doesn't change with time) and then use achieved values as the starting parameters for switching model fit.</li>
</ul>

In [1]:
import numpy as np
from statsmodels.tsa.statespace.api import MLEModel
from statsmodels.tsa.statespace.regime_switching.api import RegimeSwitchingMLEModel
from statsmodels.tsa.statespace.regime_switching.tests.results import results_kim1994

def _transform_ar_coefs(unconstrained):
    root1 = unconstrained[0] / (1 + np.abs(unconstrained[0]))
    root2 = unconstrained[1] / (1 + np.abs(unconstrained[1]))
    return (root1 + root2, -root1 * root2)

def _untransform_ar_coefs(constrained):
    b_coef = constrained[0]
    c_coef = constrained[1]
    root1 = (b_coef - np.sqrt(b_coef * b_coef + 4 * c_coef)) / 2.0
    root2 = (b_coef + np.sqrt(b_coef * b_coef + 4 * c_coef)) / 2.0
    unconstrained_roots = (root1 / (1 - np.sign(root1) * root1),
            root2 / (1 - np.sign(root2) * root2))
    return unconstrained_roots

def get_model_matrices(params):
    '''
    Transforms parameter vector into state space representation matrices.
    '''
    
    k_regimes = 2
    k_endog = 1
    k_states = 2
    k_posdef = 1

    p, q, phi_1, phi_2, sigma, delta_0, delta_1 = params

    regime_transition = np.zeros((k_regimes, k_regimes))
    regime_transition[:, :] = [[q, p], [1 - q, 1 - p]]

    design = np.zeros((k_endog, k_states, 1))
    design[0, :, 0] = [1, -1]

    obs_intercept = np.zeros((k_regimes, k_endog, 1))
    obs_intercept[:, 0, 0] = [delta_0, delta_1]

    transition = np.zeros((k_states, k_states, 1))
    transition[:, :, 0] = [[phi_1, phi_2], [1, 0]]

    selection = np.zeros((k_states, k_posdef, 1))
    selection[:, :, 0] = [[1], [0]]

    state_cov = np.zeros((k_posdef, k_posdef, 1))
    state_cov[0, 0, 0] = sigma**2

    initial_state_mean = np.zeros((k_states,))

    transition_outer_sqr = np.zeros((4, 4))

    for i in range(0, 2):
        for j in range(0, 2):
            transition_outer_sqr[i * 2:i * 2 + 2, j * 2:j * 2 + 2] = transition[i, j, 0] * transition[:, :, 0]

    nonpos_def_state_cov = selection.dot(state_cov).dot(selection.T)

    initial_state_cov_vector = np.linalg.inv(np.eye(4) -
            transition_outer_sqr).dot(nonpos_def_state_cov.reshape(-1, 1))

    initial_state_cov = initial_state_cov_vector.reshape(k_states, k_states).T

    return (regime_transition,
            design,
            obs_intercept,
            transition,
            selection,
            state_cov,
            initial_state_mean,
            initial_state_cov)

class Linear_Kim1994Model(MLEModel):
    '''
    Model without switching.
    See MLEModel documentation and examples for details.
    '''

    def transform_params(self, unconstrained):
        constrained = np.array(unconstrained)
        constrained[:2] = _transform_ar_coefs(unconstrained[:2])
        return constrained

    def untransform_params(self, constrained):
        unconstrained = np.array(constrained)
        unconstrained[:2] = _untransform_ar_coefs(constrained[:2])
        return unconstrained

    def update(self, params, *args, **kwargs):
        
        params = super(Linear_Kim1994Model, self).update(params, *args,
                **kwargs)

        _, self['design'], obs_intercept, self['transition'], \
                self['selection'], self['state_cov'], initial_state_mean, \
                initial_state_cov = get_model_matrices(np.hstack((0.5, 0.5, params, 0)))

        self['obs_intercept'] = obs_intercept[0]

        self.initialize_known(initial_state_mean, initial_state_cov)


class Kim1994Model(RegimeSwitchingMLEModel):
    '''
    Switching model.
    '''

    def __init__(self, *args, **kwargs):
        '''
        need to specify switching params
        '''

        super(Kim1994Model, self).__init__(*args, **kwargs)

        # See MarkovSwitchingParams class documentation
        self.parameters['phi'] = [False, False]
        self.parameters['sigma'] = [False]
        self.parameters['delta'] = [True]
    
    @property
    def nonswitching_model_type(self):
        # Need to know this type, since its instance will perform non-switching fitting for
        # starting parameters estimation (if fit_nonswitching_first option is set to True in fit method).
        return Linear_Kim1994Model
    
    def update_params(self, params, nonswitching_params):
        # This method is used, when non-switching fitting is done, and we need to get
        # switching model starting params.
        # nonswitching_model_params consists of 4 values: (phi_1, phi_2, sigma, delta).
        params[self.parameters['phi']] = nonswitching_params[:2]
        params[self.parameters['sigma']] = nonswitching_params[2]
        # Without shifting a little we are stuck in local maximum
        params[self.parameters[0, 'delta']] = nonswitching_params[3] - 1e-2
        params[self.parameters[1, 'delta']] = nonswitching_params[3] + 1e-2
        return params

    def get_nonswitching_params(self, params):
        # This is method is inverse to previous, it is used when we need to obtain nonswitching model
        # starting params for nonswitching fitting from params, provided by user in the fit arguments.
        # Here we use mean value as a single delta.
        nonswitching_params = np.zeros((4,), dtype=self.ssm.dtype)
        nonswitching_params[:2] = params[self.parameters['phi']]
        nonswitching_params[2] = params[self.parameters['sigma']]
        nonswitching_params[3] = params[self.parameters['delta']].mean()
        return nonswitching_params

    # Regime transition matrix, by default, uses logistic transformation, while model params
    # transformation is user-defined.
    
    def transform_model_params(self, unconstrained):

        constrained = np.array(unconstrained)
        constrained[self.parameters['phi']] = \
                _transform_ar_coefs(unconstrained[self.parameters['phi']])
        return constrained

    def untransform_model_params(self, constrained):
        unconstrained = np.array(constrained)
        unconstrained[self.parameters['phi']] = \
                _untransform_ar_coefs(constrained[self.parameters['phi']])
        return unconstrained

    def update(self, params, **kwargs):
        # params vector here contains both groups of parameters and has the size of 7:
        # (p, q, phi_1, phi_2, sigma, delta_1, delta_2)
        params = super(Kim1994Model, self).update(params, **kwargs)

        self['regime_transition'], self['design'], self['obs_intercept'], \
                self['transition'], self['selection'], self['state_cov'], \
                initial_state_mean, initial_state_cov = get_model_matrices(params)

        self.initialize_known(initial_state_mean, initial_state_cov)
        self.initialize_stationary_regime_probs()

In [4]:
k_regimes = 2
k_endog = 1
k_states = 2
k_posdef = 1

true = results_kim_filter.kim_je
data = np.array(true['data'])
data = np.log(data)*100
obs = np.array(data[1:152] - data[:151])

# Model initialization
model = Kim1994Model(k_regimes, obs, k_states, loglikelihood_burn=true['start'],
        k_posdef=k_posdef)

# Kim and Nelson use untransformed values to start from.
result_params = model.fit(start_params=np.array(true['untransformed_start_parameters']),
        transformed=False)

print(result_params)
print(model.loglike(result_params))

[ 0.04967302  0.44250441  1.26075752 -0.35336175  0.80138743 -1.29268603
  0.94561225]
-178.915785233


In [5]:
# Example of optimization, starting with parameters, which were obtained from non-switching fit.

# By default, RegimeSwitchingMLEModel uses all-ones untransformed vector to start from
result_params = model.fit(fit_nonswitching_first=True)

print(result_params)
print(model.loglike(result_params))

[ 0.05036811  0.43206027  1.30474115 -0.42558737  0.7937848  -1.35798018
  0.94656825]
-179.353125273


<h1>References</h1>
<p>Chang-Jin Kim and Charles R. Nelson<br>
"State-Space Models with Regime-Switching:
Classical and Gibbs-Sampling Approaches with Applications"</p>